Finding whether the given job posting is real based on job title and requirements




1.   0  -----> Real
2.   1  -----> Fake




In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
job_data = pd.read_csv('/content/fake_job_postings.csv')

In [ ]:
job_data.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [ ]:
job_data['fraudulent'].value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [ ]:
job_data.shape

(17880, 18)

In [ ]:
job_data.describe

<bound method NDFrame.describe of        job_id                                              title  \
0           1                                   Marketing Intern   
1           2          Customer Service - Cloud Video Production   
2           3            Commissioning Machinery Assistant (CMA)   
3           4                  Account Executive - Washington DC   
4           5                                Bill Review Manager   
...       ...                                                ...   
17875   17876                   Account Director - Distribution    
17876   17877                                 Payroll Accountant   
17877   17878  Project Cost Control Staff Engineer - Cost Con...   
17878   17879                                   Graphic Designer   
17879   17880                         Web Application Developers   

                   location   department salary_range  \
0          US, NY, New York    Marketing          NaN   
1            NZ, , Auckland      Su

In [ ]:
job_data.isnull().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [ ]:
job_data = job_data.fillna('')

In [ ]:
job_data.isnull().sum()

job_id                 0
title                  0
location               0
department             0
salary_range           0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
dtype: int64

Merging the Job Title and its requirements


In [ ]:
job_data['Title_Requirements'] = job_data['title']+ ''+ job_data['requirements']

In [ ]:
print(job_data['Title_Requirements'])

0        Marketing InternExperience with content manage...
1        Customer Service - Cloud Video ProductionWhat ...
2        Commissioning Machinery Assistant (CMA)Impleme...
3        Account Executive - Washington DCEDUCATION: Ba...
4        Bill Review ManagerQUALIFICATIONS:RN license i...
                               ...                        
17875    Account Director - Distribution To ace this ro...
17876    Payroll Accountant- B.A. or B.S. in Accounting...
17877    Project Cost Control Staff Engineer - Cost Con...
17878    Graphic Designer1. Must be fluent in the lates...
17879    Web Application DevelopersWe want to hear from...
Name: Title_Requirements, Length: 17880, dtype: object


Separating the data and label

In [ ]:
x = job_data.drop(columns = 'fraudulent', axis = 1)
y = job_data['fraudulent']

In [ ]:
print(x)
print(y)

       job_id                                              title  \
0           1                                   Marketing Intern   
1           2          Customer Service - Cloud Video Production   
2           3            Commissioning Machinery Assistant (CMA)   
3           4                  Account Executive - Washington DC   
4           5                                Bill Review Manager   
...       ...                                                ...   
17875   17876                   Account Director - Distribution    
17876   17877                                 Payroll Accountant   
17877   17878  Project Cost Control Staff Engineer - Cost Con...   
17878   17879                                   Graphic Designer   
17879   17880                         Web Application Developers   

                   location   department salary_range  \
0          US, NY, New York    Marketing                
1            NZ, , Auckland      Success                
2           

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Stemming

In [ ]:
#Stemming is the process of reducing a word to its root word

In [ ]:
Porter_stem = PorterStemmer()

In [ ]:
def stemming(Title_Requirements):
  stemmed_data = re.sub('[^a-zA-z]','',Title_Requirements)
  stemmed_data = stemmed_data.lower()
  stemmed_data = stemmed_data.split()
  stemmed_data = [Porter_stem.stem(word) for word in stemmed_data if not word in stopwords.words('english')]
  stemmed_data = ''.join(stemmed_data)
  return stemmed_data

In [ ]:
z = job_data['Title_Requirements']

In [ ]:
z = z.apply(stemming)

In [ ]:
print(z)

0        marketinginternexperiencewithcontentmanagement...
1        customerservicecloudvideoproductionwhatweexpec...
2        commissioningmachineryassistantcmaimplementpre...
3        accountexecutivewashingtondceducationbachelors...
4        billreviewmanagerqualificationsrnlicenseinthes...
                               ...                        
17875    accountdirectordistributiontoacethisroleyouwil...
17876    payrollaccountantbaorbsinaccountingdesiretohav...
17877    projectcostcontrolstaffengineercostcontrolexpt...
17878    graphicdesignermustbefluentinthelatestversions...
17879    webapplicationdeveloperswewanttohearfromyouify...
Name: Title_Requirements, Length: 17880, dtype: object


In [ ]:
x = z.values
y = job_data['fraudulent'].values

In [ ]:
print(x)

['marketinginternexperiencewithcontentmanagementsystemsamajorplusanybloggingcountsfamiliarwiththefoodeditorialvoiceandaestheticlovesfoodappreciatestheimportanceofhomecookingandcookingwiththeseasonsmeticulouseditorperfectionistobsessiveattentiontodetailmaddenedbytyposandbrokenlinksdelightedbyfindingandfixingthemcheerfulunderpressureexcellentcommunicationskillsamultitaskerandjugglerofresponsibilitiesbigandsmallinterestedinandengagedwithsocialmedialiketwitterfacebookandpinterestlovesproblemsolvingandcollaboratingtodrivefoodforwardthinksbigpicturebutpitchesinonthenittygrittyofrunningasmallcompanydishesshoppingadministrativesupportcomfortablewiththerealitiesofworkingforastartupbeingoncalloneveningsandweekendsandworkinglonghour'
 'customerservicecloudvideoproductionwhatweexpectfromyouyourkeyresponsibilitywillbetocommunicatewiththeclientsecondsteamandfreelancecommunitythroughoutthevideoproductionprocessincludingshootplanningsecuringfreelancetalentmanagingworkflowandtheonlineproductionmanageme

In [ ]:
print(y)

[0 0 0 ... 0 0 0]


In [ ]:
y.shape

(17880,)

In [ ]:
#Converting textual data into numerical data or feature vectors


In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit(x)

TfidfVectorizer()

In [ ]:
x = vectorizer.transform(x)

In [ ]:
print(x)

  (0, 7796)	1.0
  (1, 2813)	1.0
  (2, 2059)	1.0
  (3, 124)	1.0
  (4, 1111)	1.0
  (5, 148)	1.0
  (6, 5528)	1.0
  (7, 7197)	1.0
  (8, 5847)	1.0
  (9, 2728)	1.0
  (10, 786)	1.0
  (11, 13303)	1.0
  (12, 696)	1.0
  (13, 6202)	1.0
  (14, 119)	1.0
  (15, 14196)	1.0
  (16, 5489)	1.0
  (17, 12755)	1.0
  (18, 14121)	1.0
  (19, 9474)	1.0
  (20, 7699)	1.0
  (21, 4924)	1.0
  (22, 4140)	1.0
  (23, 14082)	1.0
  (24, 2614)	1.0
  :	:
  (17855, 638)	1.0
  (17856, 14429)	1.0
  (17857, 1808)	1.0
  (17858, 9202)	1.0
  (17859, 5994)	1.0
  (17860, 12962)	1.0
  (17861, 6946)	1.0
  (17862, 1946)	1.0
  (17863, 6008)	1.0
  (17864, 8429)	1.0
  (17865, 9367)	1.0
  (17866, 9380)	1.0
  (17867, 350)	1.0
  (17868, 1985)	1.0
  (17869, 12971)	1.0
  (17870, 11553)	1.0
  (17871, 14273)	1.0
  (17872, 9651)	1.0
  (17873, 10346)	1.0
  (17874, 6759)	1.0
  (17875, 50)	1.0
  (17876, 9092)	1.0
  (17877, 9850)	1.0
  (17878, 5370)	1.0
  (17879, 14305)	1.0


Splitting the data into training and test data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,stratify = y, random_state = 3)

In [ ]:
print(x.shape,x_train.shape,x_test.shape)

(17880, 14692) (14304, 14692) (3576, 14692)


Training the Model : Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train,y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [ ]:
#Printing the accuracy score of Training data

In [ ]:
x_train_prediction = model.predict(x_train)
Training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('The accuracy score of training data is :',Training_data_accuracy)

The accuracy score of training data is : 0.9577041387024608


In [ ]:
#Printing the accuracy score of testing data
x_test_prediction = model.predict(x_test)
Test_data_accuracy = accuracy_score(x_test_prediction,y_test)

In [ ]:
print('The accuracy score of test data is:',Test_data_accuracy)

The accuracy score of test data is: 0.956096196868009


Prediction System

In [ ]:
x_input = x_test[1784]

prediction = model.predict(x_input)
print(prediction)
if [prediction[0]==0]:
  print("The job posting is Real")
else:
  print('The job posting is Fake')

[0]
The job posting is Real
